# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('simpsons_dataset.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download'
        output = 'songs_dataset.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['dolly-parton.txt',
 'cake.txt',
 'britney-spears.txt',
 'paul-simon.txt',
 'beatles.txt',
 'nickelback.txt',
 'rihanna.txt',
 'disney.txt',
 'kanye.txt',
 'drake.txt',
 'kanye-west.txt',
 'jimi-hendrix.txt',
 'nicki-minaj.txt',
 'janisjoplin.txt',
 'alicia-keys.txt',
 'bieber.txt',
 'bob-dylan.txt',
 'amy-winehouse.txt',
 'ludacris.txt',
 'bruno-mars.txt',
 'lin-manuel-miranda.txt',
 'r-kelly.txt',
 'eminem.txt',
 'bob-marley.txt',
 'notorious_big.txt',
 'dr-seuss.txt',
 'dj-khaled.txt',
 'missy-elliott.txt',
 'bruce-springsteen.txt',
 'adele.txt',
 'lady-gaga.txt',
 'patti-smith.txt',
 'joni-mitchell.txt',
 'lil-wayne.txt',
 'lorde.txt',
 'nursery_rhymes.txt',
 'prince.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'bjork.txt',
 'leonard-cohen.txt',
 'notorious-big.txt',
 'Lil_Wayne.txt',
 'nirvana.txt',
 'radiohead.txt',
 'johnny-cash.txt',
 'blink-182.txt',
 'dickinson.txt',
 'Kanye_West.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bruno-mars.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0
0,Now greetings to the world! Standing at this l...
1,"Whiskey coming through my pores,"
2,Feeling like I run this whole block.
3,Lotto tickets cheap beer
4,"That's why you can catch me here,"


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3270


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[427:435]

[['no', 'no', 'no', 'no', 'this', 'hit', 'that', 'ice', 'cold'],
 ['michelle', 'pfeiffer', 'that', 'white', 'gold'],
 ['this', 'one', 'for', 'them', 'hood', 'girls'],
 ['them', 'good', 'girls', 'straight', 'masterpieces'],
 ["stylin'", 'whilen', "livin'", 'it', 'up', 'in', 'the', 'city'],
 ['got', 'chucks', 'on', 'with', 'saint', 'laurent'],
 ['got',
  'kiss',
  'myself',
  "i'm",
  'so',
  'pretty',
  "i'm",
  'too',
  'hot',
  'hot',
  'damn'],
 ['called', 'a', 'police', 'and', 'a', 'fireman']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 3270


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 678


In [22]:
w2v_model.wv.vocab

{"'cause": <gensim.models.keyedvectors.Vocab at 0x7f5d84d497d0>,
 "'til": <gensim.models.keyedvectors.Vocab at 0x7f5d84d4c310>,
 '3x': <gensim.models.keyedvectors.Vocab at 0x7f5d84ce7250>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7f5d84d49990>,
 'about': <gensim.models.keyedvectors.Vocab at 0x7f5d84d528d0>,
 'afraid': <gensim.models.keyedvectors.Vocab at 0x7f5d84d599d0>,
 'again': <gensim.models.keyedvectors.Vocab at 0x7f5d84d4cc90>,
 'agree': <gensim.models.keyedvectors.Vocab at 0x7f5d84d4ef10>,
 'ah': <gensim.models.keyedvectors.Vocab at 0x7f5d84d60210>,
 'ahh': <gensim.models.keyedvectors.Vocab at 0x7f5d84d62e10>,
 "ain't": <gensim.models.keyedvectors.Vocab at 0x7f5d84d49950>,
 'aint': <gensim.models.keyedvectors.Vocab at 0x7f5d84d62c90>,
 'air': <gensim.models.keyedvectors.Vocab at 0x7f5d84d52a50>,
 'all': <gensim.models.keyedvectors.Vocab at 0x7f5d84d4e750>,
 'alone': <gensim.models.keyedvectors.Vocab at 0x7f5d84d59690>,
 'along': <gensim.models.keyedvectors.Vocab at 0x7f5d84d

### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 225532.78125
Loss after epoch 1: 143554.96875
Loss after epoch 2: 141304.21875
Loss after epoch 3: 145517.40625
Loss after epoch 4: 141690.4375
Loss after epoch 5: 136914.625
Loss after epoch 6: 127180.9375
Loss after epoch 7: 109596.75
Loss after epoch 8: 104740.625
Loss after epoch 9: 102631.125
Loss after epoch 10: 98675.0
Loss after epoch 11: 96093.625
Loss after epoch 12: 93729.625
Loss after epoch 13: 91034.25
Loss after epoch 14: 89253.75
Loss after epoch 15: 87733.625
Loss after epoch 16: 86131.5
Loss after epoch 17: 85297.75
Loss after epoch 18: 79307.75
Loss after epoch 19: 77399.25
Loss after epoch 20: 77704.0
Loss after epoch 21: 75957.75
Loss after epoch 22: 75520.0
Loss after epoch 23: 75884.5
Loss after epoch 24: 75135.25
Loss after epoch 25: 75236.75
Loss after epoch 26: 74138.25
Loss after epoch 27: 73890.25
Loss after epoch 28: 73727.0
Loss after epoch 29: 72680.5
Loss after epoch 30: 74113.25
Loss after epoch 31: 73990.25
Loss after epoch 32: 7230

(820625, 1345000)

### 4 - Ensayar

In [25]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["liquor"], topn=10)

[('store', 0.8884459733963013),
 ('blues', 0.713279664516449),
 ('higher', 0.6362438797950745),
 ('shot', 0.6126545071601868),
 ('voices', 0.6095480918884277),
 ('favourite', 0.5944533348083496),
 ('stares', 0.5807141065597534),
 ('part', 0.5764656066894531),
 ('distance', 0.5751705169677734),
 ('chance', 0.5734277963638306)]

In [24]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["heaven"], topn=10)

[("gon'", -0.09452524036169052),
 ('say', -0.10013270378112793),
 ('put', -0.14427734911441803),
 ('goodbye', -0.15772822499275208),
 ('again', -0.1589554250240326),
 ('ask', -0.16355621814727783),
 ('do', -0.1636718511581421),
 ('who', -0.1663840413093567),
 ("let's", -0.16666346788406372),
 ('was', -0.1676674485206604)]

In [34]:
# Palabras que MAS se relacionan con...:
w2v_model.wv.most_similar(positive=["mom"], topn=10)

[('favorite', 0.8822276592254639),
 ('best', 0.6266688108444214),
 ('press', 0.5960018634796143),
 ('higher', 0.5903213024139404),
 ('lucky', 0.5778060555458069),
 ('sing', 0.5649850368499756),
 ('true', 0.5616679191589355),
 ('promise', 0.5542293787002563),
 ('friend', 0.5536810755729675),
 ('because', 0.5524802803993225)]

In [23]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["diamonds"], topn=10)

[('sheets', 0.9740362167358398),
 ('silk', 0.9482098817825317),
 ('white', 0.874255895614624),
 ('lovelies', 0.7435137033462524),
 ('roll', 0.6556969881057739),
 ('gave', 0.6503945589065552),
 ('hours', 0.6384258270263672),
 ('stops', 0.6323196291923523),
 ('calling', 0.6246854662895203),
 ('stares', 0.610127329826355)]

In [27]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["strawberry"], topn=5)

[('champagne', 0.9961233735084534),
 ('ice', 0.9105430841445923),
 ('ohhh', 0.8022201061248779),
 ('nude', 0.6799787878990173),
 ('pinky', 0.676993727684021)]

In [36]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["veins"])

[('shoot', 0.9529938101768494),
 ('brain', 0.7390738725662231),
 ('voices', 0.7191059589385986),
 ('tears', 0.7164756059646606),
 ('rest', 0.6844604015350342),
 ('bed', 0.6818910837173462),
 ('without', 0.6600667834281921),
 ('finesse', 0.6587929725646973),
 ('favourite', 0.6511949300765991),
 ('hate', 0.634775698184967)]

### 5 - Visualizar agrupación de vectores

In [20]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [21]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab
